In [1]:
!pip install accelerate

In [2]:
!pip install pandas transformers guidance

In [3]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [4]:
pip install einops

In [5]:
#imports
import json
from transformers import AutoTokenizer
import transformers
import torch
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

import warnings
from typing import Any, Dict, Tuple
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import warnings

warnings.filterwarnings("ignore", category=UserWarning, message="A new version of the following files was downloaded from")

In [6]:
!pip install numpy

In [7]:
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
import numpy as np
from guidance import gen
import os

import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##### Balance of Test Datasets

In [19]:
# Load the dataset
with open('/content/drive/MyDrive/Master Project-Hallucination/dataset/Paraphrasing/SHROOM_trial-v1.1/trial-v1.json', 'r') as f:
    data = json.load(f)

In [20]:
import pandas as pd
# Convert the data to a DataFrame
df = pd.DataFrame(data)

In [21]:
df['label'].value_counts()

label
Hallucination        42
Not Hallucination    38
Name: count, dtype: int64

In [34]:
df_new = pd.read_csv('/content/drive/MyDrive/Master Project-Hallucination/dataset/HalluciGen-PG/data/trial_en.csv')

In [35]:
df_new

,id,source,type,hyp1,hyp2,label
0,0,The population has declined in some 210 of the...,antonym,In the majority of Sweden's 280 municipalities...,In the majority of Sweden's 280 municipalities...,hyp1
1,1,The draft agenda as drawn up by the Conference...,negation,The Conference of Presidents hasn't distribute...,The Conference of Presidents has distributed t...,hyp1
2,2,I am always grateful for comments and suggesti...,antonym,I think I need to listen to everyone's views i...,I think I need to listen to everyone's views i...,hyp1
3,3,The fact is that a key omission from the propo...,named entity,Agenda 2030 does not include a chapter on rene...,Agenda 2000 does not include a chapter on rene...,hyp1
4,4,"We need quite specific legislative proposals, ...",natural,Legislative proposals based on the Treaty of A...,Legislative proposals based solely on the Trea...,hyp2
5,5,This state of affairs has not changed in more ...,addition,There has been no change in the status quo in ...,The state of affairs is1-65561-65561-65561-655...,hyp2
6,6,"Madam President, I am speaking on behalf of ou...",gender,One of the motions for a resolution was drafte...,One of the motions for a resolution was drafte...,hyp2
7,7,Amendment No 1 in the French version deletes i...,natural,The French version excludes the expression'pol...,The French version excludes the expression 'po...,hyp1
8,8,The European Commission proposes that this inf...,number,The EU wants this information to enter into fo...,The EU wants this information to enter into fo...,hyp2
9,9,"Mr President, I did prepare a speech but I hav...",pronoun,Many of the points they were going to make in ...,Many of the points I was going to make in my s...,hyp1


In [36]:
df_new['label'].value_counts()

label
hyp1    9
hyp2    7
Name: count, dtype: int64

In [38]:
df_new = pd.read_csv('/content/drive/MyDrive/Master Project-Hallucination/dataset/HalluciGen-PG/data/trial_sv.csv')
df_new['label'].value_counts()

label
hyp2    11
hyp1     9
Name: count, dtype: int64

In [57]:
df_new = pd.read_json('/content/drive/MyDrive/Master Project-Hallucination/dataset/HalluciGen-Translation/de-en/trial.de-en.jsonl', lines = True)
df_new['label'].value_counts()

label
hyp2    5
hyp1    5
Name: count, dtype: int64

In [63]:
df_new = pd.read_json('/content/drive/MyDrive/Master Project-Hallucination/dataset/HalluciGen-Translation/de-en/trial.en-de.jsonl', lines = True)
df_new['label'].value_counts()

label
hyp2    7
hyp1    3
Name: count, dtype: int64

In [60]:
df_new = pd.read_json('/content/drive/MyDrive/Master Project-Hallucination/dataset/HalluciGen-Translation/fr-en/trial.en-fr.jsonl', lines = True)
df_new['label'].value_counts()

label
hyp1    7
hyp2    3
Name: count, dtype: int64

In [61]:
df_new = pd.read_json('/content/drive/MyDrive/Master Project-Hallucination/dataset/HalluciGen-Translation/fr-en/trial.fr-en.jsonl', lines = True)
df_new['label'].value_counts()

label
hyp2    7
hyp1    3
Name: count, dtype: int64

In [70]:
df_new = pd.read_json('/content/drive/MyDrive/Master Project-Hallucination/dataset/HalluciGen-Translation/de-en/test_detection.de-en.jsonl', lines = True)
df_new

,id,langpair,source,hyp1,hyp2
0,0,de-en,In Ländern ist das Anbieten von Luxus-Gästehäu...,"In countries, offering luxury guesthouses has ...","In developed countries, offering luxury guesth..."
1,1,de-en,"Dies ist in Großbritannien gängige Praxis, abe...","This is common practice in the UK, but Scottis...","This is common practice elsewhere in the UK, b..."
2,2,de-en,„Auf Grundlage dieses Fossils lässt sich sagen...,"“Based on this fossil, we can say that the sep...","“Based on this fossil, we can say that the sep..."
3,3,de-en,"Wir werden niemals ein Auto überholen, einen B...","We will never overtake a car, build a well in ...","We will never overtake a car, build a well in ..."
4,4,de-en,"Ein Minenabwehrschiff der Avenger-Klasse, das ...","An Avenger class anti-mining ship, the ship wa...","An Avenger class anti-mining ship, the ship wa..."
...,...,...,...,...,...
95,95,de-en,Die Anzahl der gemeldeten Brände betrug Anfang...,The number of reported fires at the ending of ...,The number of reported fires in early February...
96,96,de-en,Der Ponoru ist eine Nebenfluss des Horezu in R...,The Ponoru River is a distributary of the Hore...,The Ponoru River is a tributary of the Horezu ...
97,97,de-en,Leslie Waddington eröffnete 1966 seine eigene ...,"In 1966, started his own gallery in London's C...","In 1966, he started his own gallery at Cork St..."
98,98,de-en,In der Zwischenzeit kaufte die Air Force die S...,"In the meantime, the Air Force purchased the S...","In the meantime, the Air Force sold the SR71, ..."


In [71]:
df_new = pd.read_json('/content/drive/MyDrive/Master Project-Hallucination/dataset/HalluciGen-Translation/de-en/test_generation.de-en.jsonl', lines = True)
df_new

,id,langpair,source
0,0,de-en,In Ländern ist das Anbieten von Luxus-Gästehäu...
1,1,de-en,"Dies ist in Großbritannien gängige Praxis, abe..."
2,2,de-en,„Auf Grundlage dieses Fossils lässt sich sagen...
3,3,de-en,"Wir werden niemals ein Auto überholen, einen B..."
4,4,de-en,"Ein Minenabwehrschiff der Avenger-Klasse, das ..."
...,...,...,...
95,95,de-en,Die Anzahl der gemeldeten Brände betrug Anfang...
96,96,de-en,Der Ponoru ist eine Nebenfluss des Horezu in R...
97,97,de-en,Leslie Waddington eröffnete 1966 seine eigene ...
98,98,de-en,In der Zwischenzeit kaufte die Air Force die S...


### Add "binary_label"
"Not Hallucination" = 0 and "Hallucination" = 1

In [49]:
for item in data:
    if item['label'] == 'Hallucination':
        item['binary_label'] = 1
    else:
        item['binary_label'] = 0

# Falcon

In [42]:
from guidance import models
falcon = models.Transformers("tiiuae/falcon-7b-instruct",echo=False)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [54]:
def predict_hallucination(data, source, hypothesis):
    # Generate the prompt
    prompt = "Given a Text and one hypothesis determine if the hypothesis is 'Hallucination' (1) or 'Not Hallucination' (0). Here are some examples:"
    prompt_initial_length= len(prompt)
    for i in range(4):
      prompt += f'\nExample {i+1}:\nText: {data[i]["src"]}\nHypothesis: {data[i]["hyp"]}\nLabel: {data[i]["binary_label"]}'
    prompt_length_text = len(prompt)
    prompt += "\nText: " + source + "\nHypothesis: "+ hypothesis + "\nLabel:"

    # define generated output as 'gen_number'
    prediction = falcon + prompt + gen('gen_number', regex='[0-1]+', max_tokens=1, stop=',')
    # Return 'Hallucination' if the prediction is 1, 'Not Hallucination' otherwise
    print(prediction['gen_number'])
    return prediction['gen_number'], prompt, prompt_initial_length


## Evaluation

In [44]:
def save_confusion_matrix(data, index, df_result):
  label_true = [item['binary_label'] for item in data[4:] if 'binary_label' in item]
  label_pred = list(map(int, df_result["prediction"][4:].tolist()))

  conf_matrix = confusion_matrix(label_true, label_pred)
  cm_display = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=[0, 1])
  fig, ax = plt.subplots()
  cm_display.plot(ax=ax)
  plt.savefig(f'/content/drive/MyDrive/Master Project-Hallucination/Group1_results/falcon2/{index}_confusion_matrix.png')
  plt.show()


In [45]:
def calculate_evaluation(data, df_result):
  label_true = [item['binary_label'] for item in data[4:] if 'binary_label' in item]
  label_pred = list(map(int, df_result["prediction"][4:].tolist()))
  accuracy = accuracy_score(label_true, label_pred)
  f1 =f1_score(label_true, label_pred)
  return accuracy, f1


In [46]:
import os
def save_results(accuracy, f1, data, df_result):
  if os.path.isfile('/content/drive/MyDrive/Master Project-Hallucination/Group1_results/falcon2/results.csv'):
    df = pd.read_csv('/content/drive/MyDrive/Master Project-Hallucination/Group1_results/falcon2/results.csv')
    new_row = {'prompt': prompt[:prompt_initial_length], 'accuracy': accuracy, 'f1 score': f1}
    # Append the dictionary to the DataFrame
    df.loc[len(df)] = new_row
    index = len(df) - 1
    print(index)
    save_confusion_matrix(data, index, df_result)
    df.to_csv('/content/drive/MyDrive/Master Project-Hallucination/Group1_results/falcon2/results.csv', index=False)

  else:
    d = {'prompt': [prompt[:prompt_initial_length]], 'accuracy': [accuracy], 'f1 score': [f1]}
    df = pd.DataFrame(data=d)
    df.to_csv('/content/drive/MyDrive/Master Project-Hallucination/Group1_results/falcon2/results.csv')
    index = 0
    save_confusion_matrix(data, index, df_result)


### Chain of Thought and reasoning steps

In [55]:
def predict_hallucination(data, source, hypothesis):
    # Generate the prompt
    prompt = "Imagine you are a classifier. Given a Text and one hypothesis. Your task is to classify if the hypothesis is 'Hallucination' (1) or 'Not Hallucination' (0). Explain your choice. Here are some examples:"
    prompt_inital_length= len(prompt)
    prompt += f'\nExample {0}:\nText: {data[0]["src"]}\nHypothesis: {data[0]["hyp"]}\nLabel: {data[0]["binary_label"]}'
    prompt += "The hypothesis 'A district of Kowloon, China' is considered a hallucination in this context because it misinterprets the information in the source text. The text mentions a restaurant called The City Chiuchow, which is famous for its goose dishes and specialities from the Chiuchow region. However, there is no indication in the text that Kowloon refers to a district. Kowloon is actually a district within Hong Kong, not a separate district within China. Therefore, the assumption that Kowloon is a district is a misinterpretation or hallucination, as it deviates from the reality presented in the text."
    prompt += f'\nExample {1}:\nText: {data[1]["src"]}\nHypothesis: {data[1]["hyp"]}\nLabel: {data[1]["binary_label"]}'
    prompt += "The hypothesis '(Wicca) A witchdoctor' is considered a hallucination in this context because it inaccurately interprets the information provided in the source text. The source text mentions visiting a witchdoctor, referred to as 'igqira,' who communicates with the spirits of the ancestors, known as izinyanya. The term witchdoctor typically refers to a traditional healer or spiritual practitioner in various cultures, especially in African or indigenous communities, and is not specific to Wicca. Therefore, assuming that the witchdoctor mentioned in the text is associated with Wicca is a misinterpretation or hallucination, as it deviates from the cultural and contextual information provided."
    prompt += f'\nExample {2}:\nText: {data[2]["src"]}\nHypothesis: {data[2]["hyp"]}\nLabel: {data[2]["binary_label"]}'
    prompt += "The hypothesis '(nautical) A halyard' is considered not hallucination in this context because it accurately interprets the terminology used in the source text. The text mentions 'halyard' implicitly by describing the action of 'keeping strain on both parts of the halyard' to ensure that the 'pigstick' remains vertical. A halyard is a rope or tackle used for hoisting and lowering sails, flags, or other objects on a ship. Since the term 'halyard' is directly related to nautical terminology and fits the context of the action described in the text, the hypothesis aligns with the information provided and is therefore considered accurate rather than a hallucination."
    prompt += f'\nExample {3}:\nText: {data[3]["src"]}\nHypothesis: {data[3]["hyp"]}\nLabel: {data[3]["binary_label"]}'
    prompt += "The hypothesis 'The quality of being rational' is not deemed a hallucination in this context because it aligns with the content of the source text."
    prompt_length_text = len(prompt)
    prompt += "\nText: " + source + "\nHypothesis: "+ hypothesis + "\nLabel:"

    # define generated output as 'gen_number'
    prediction = falcon + prompt + gen('gen_number', regex='[0-1]+', max_tokens=1, stop=',')
    # Return 'Hallucination' if the prediction is 1, 'Not Hallucination' otherwise
    #print(prediction['gen_number'])
    return prediction['gen_number'], prompt, prompt_initial_length

In [56]:
# Create a new column 'prediction' and initialize it with NaN
df_result_falcon = df
df_result_falcon['prediction'] = np.nan

# Loop over the DataFrame
for i, row in df_result_falcon.iterrows():
    # Apply the function and store the result in the 'prediction' column
    prediction, prompt, prompt_initial_length = predict_hallucination(data, row['src'], row['hyp'])
    df_result_falcon.at[i, 'prediction'] = prediction

In [ ]:
df_result_falcon

In [ ]:
accuracy, f1 = calculate_evaluation(data)

In [ ]:
save_results(accuracy, f1, data)

# LLAMA

In [62]:
#load model Llama2
model = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [64]:
torch.manual_seed(0)
#prompting each item in dataset and getting the predicted label for hallucination
for x in data[4:]:
  prompt = "Imagine you are a classifier. Given a Text and one hypothesis. Your task is to classify if the hypothesis is 'Hallucination' (1) or 'Not Hallucination' (0). Explain your choice. Here are some examples:"
  prompt_inital_length= len(prompt)
  prompt += f'\nExample {0}:\nText: {data[0]["src"]}\nHypothesis: {data[0]["hyp"]}\nLabel: {data[0]["binary_label"]}'
  prompt += "The hypothesis 'A district of Kowloon, China' is considered a hallucination in this context because it misinterprets the information in the source text. The text mentions a restaurant called The City Chiuchow, which is famous for its goose dishes and specialities from the Chiuchow region. However, there is no indication in the text that Kowloon refers to a district. Kowloon is actually a district within Hong Kong, not a separate district within China. Therefore, the assumption that Kowloon is a district is a misinterpretation or hallucination, as it deviates from the reality presented in the text."
  prompt += f'\nExample {1}:\nText: {data[1]["src"]}\nHypothesis: {data[1]["hyp"]}\nLabel: {data[1]["binary_label"]}'
  prompt += "The hypothesis '(Wicca) A witchdoctor' is considered a hallucination in this context because it inaccurately interprets the information provided in the source text. The source text mentions visiting a witchdoctor, referred to as 'igqira,' who communicates with the spirits of the ancestors, known as izinyanya. The term witchdoctor typically refers to a traditional healer or spiritual practitioner in various cultures, especially in African or indigenous communities, and is not specific to Wicca. Therefore, assuming that the witchdoctor mentioned in the text is associated with Wicca is a misinterpretation or hallucination, as it deviates from the cultural and contextual information provided."
  prompt += f'\nExample {2}:\nText: {data[2]["src"]}\nHypothesis: {data[2]["hyp"]}\nLabel: {data[2]["binary_label"]}'
  prompt += "The hypothesis '(nautical) A halyard' is considered not hallucination in this context because it accurately interprets the terminology used in the source text. The text mentions 'halyard' implicitly by describing the action of 'keeping strain on both parts of the halyard' to ensure that the 'pigstick' remains vertical. A halyard is a rope or tackle used for hoisting and lowering sails, flags, or other objects on a ship. Since the term 'halyard' is directly related to nautical terminology and fits the context of the action described in the text, the hypothesis aligns with the information provided and is therefore considered accurate rather than a hallucination."
  prompt += f'\nExample {3}:\nText: {data[3]["src"]}\nHypothesis: {data[3]["hyp"]}\nLabel: {data[3]["binary_label"]}'
  prompt += "The hypothesis 'The quality of being rational' is not deemed a hallucination in this context because it aligns with the content of the source text."

  prompt_length_text = len(prompt)
  prompt += "\nText: " + x['src'] + "\nHypothesis: "+ x['hyp'] + "\nLabel:"

  prompt_length_label = len(prompt)
  sequences = pipeline(
      prompt,
      max_new_tokens=50,
      num_return_sequences=1,
      do_sample=True,
      top_k=10,
  )
  generated_label = sequences[0]['generated_text'][prompt_length_label:].strip()
  #print(generated_label[0])
  x["predicted_label"] = int(generated_label[0])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1157: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [68]:
df_result_llama = pd.DataFrame(data)

In [69]:
df_result_llama

,hyp,ref,src,tgt,model,task,labels,label,p(Hallucination),binary_label,predicted_label
0,"A district of Kowloon, China.",tgt,The City <define> Chiuchow </define> is Kowloo...,The Chaoshan region where the Teochew dialect ...,,DM,"[Hallucination, Hallucination, Hallucination]",Hallucination,1.000000,1,NaN
1,(Wicca) A witchdoctor.,tgt,"“ I went to see the witchdoctor , the <define>...",(South Africa) A traditional tribal diviner or...,,DM,"[Hallucination, Hallucination, Not Hallucination]",Hallucination,0.666667,1,NaN
2,(nautical) A halyard.,tgt,"Haul away , keeping strain on both parts of th...",(nautical) A staff that carries a flag or penn...,,DM,"[Hallucination, Not Hallucination, Not Halluci...",Not Hallucination,0.333333,0,NaN
3,The quality of being rational.,tgt,And what difference is there between the credi...,(archaic) The quality or state of being rational.,,DM,"[Not Hallucination, Not Hallucination, Not Hal...",Not Hallucination,0.000000,0,NaN
4,(uncountable) The study of trees.,tgt,It is now generally supposed that the forbidde...,The worship of trees.,,DM,"[Hallucination, Hallucination, Hallucination]",Hallucination,1.000000,1,1.0
...,...,...,...,...,...,...,...,...,...,...,...
75,I can't do it alone.,either,Je ne peux le faire seule. Il vous faut m'aider.,I can't do it alone. You have to help me.,,MT,"[Not Hallucination, Not Hallucination, Halluci...",Not Hallucination,0.333333,0,1.0
76,He lost what he received from his father.,either,Il a perdu ce qu'il a reçu de son père.,He wasted that which he had received from his ...,,MT,"[Not Hallucination, Not Hallucination, Not Hal...",Not Hallucination,0.000000,0,1.0
77,We've discovered a lot of news.,either,Nous avons découvert une grande nouveauté.,We have discovered a great novelty.,,MT,"[Hallucination, Not Hallucination, Hallucination]",Hallucination,0.666667,1,1.0
78,Why is everyone okay?,either,Pourquoi est-ce que tout le monde rigole ?,Why is everyone laughing?,,MT,"[Hallucination, Hallucination, Hallucination]",Hallucination,1.000000,1,1.0


In [66]:
df_result_falcon

,hyp,ref,src,tgt,model,task,labels,label,p(Hallucination),prediction
0,"A district of Kowloon, China.",tgt,The City <define> Chiuchow </define> is Kowloo...,The Chaoshan region where the Teochew dialect ...,,DM,"[Hallucination, Hallucination, Hallucination]",Hallucination,1.000000,1
1,(Wicca) A witchdoctor.,tgt,"“ I went to see the witchdoctor , the <define>...",(South Africa) A traditional tribal diviner or...,,DM,"[Hallucination, Hallucination, Not Hallucination]",Hallucination,0.666667,1
2,(nautical) A halyard.,tgt,"Haul away , keeping strain on both parts of th...",(nautical) A staff that carries a flag or penn...,,DM,"[Hallucination, Not Hallucination, Not Halluci...",Not Hallucination,0.333333,1
3,The quality of being rational.,tgt,And what difference is there between the credi...,(archaic) The quality or state of being rational.,,DM,"[Not Hallucination, Not Hallucination, Not Hal...",Not Hallucination,0.000000,1
4,(uncountable) The study of trees.,tgt,It is now generally supposed that the forbidde...,The worship of trees.,,DM,"[Hallucination, Hallucination, Hallucination]",Hallucination,1.000000,1
...,...,...,...,...,...,...,...,...,...,...
75,I can't do it alone.,either,Je ne peux le faire seule. Il vous faut m'aider.,I can't do it alone. You have to help me.,,MT,"[Not Hallucination, Not Hallucination, Halluci...",Not Hallucination,0.333333,1
76,He lost what he received from his father.,either,Il a perdu ce qu'il a reçu de son père.,He wasted that which he had received from his ...,,MT,"[Not Hallucination, Not Hallucination, Not Hal...",Not Hallucination,0.000000,1
77,We've discovered a lot of news.,either,Nous avons découvert une grande nouveauté.,We have discovered a great novelty.,,MT,"[Hallucination, Not Hallucination, Hallucination]",Hallucination,0.666667,1
78,Why is everyone okay?,either,Pourquoi est-ce que tout le monde rigole ?,Why is everyone laughing?,,MT,"[Hallucination, Hallucination, Hallucination]",Hallucination,1.000000,1
